In [ ]:
# pipelin example
from sklearn.datasets import load_boston
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 데이터 로딩
load_data = load_boston()
print(type(load_data))
print(load_data.keys())
print(load_data.DESCR)

# np에서 pd로 변환하기
data = load_data['data']
df_X = pd.DataFrame(columns=load_data['feature_names'],
                   data=load_data['data'])
print(df_X.info())
# print(load_data['target'])

df_y = pd.DataFrame({'target':load_data['target']})
df = pd.concat([df_X,df_y], axis=1)


# 데이터 확인
print(df.info()) # 데이터 구조 확인
print(df.describe()) # 데이터 통계 


# 데이터 분포 확인 시각화
df.hist(figsize=(20, 30))
# plt.show()

# 상관관계 분석 및 시각화
plt.figure(figsize=(15,15))
sns.heatmap(data=df.corr(), annot=True, 
            fmt = '.2f', linewidths=.5, cmap='Blues')
# plt.show()

# df['index_check'] = [i for i in range(len(df))]
# df.plot.scatter(x='index_check', y='target')
# # plt.show()
# print(df.iloc[3:5])

# train_test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.3, random_state=2012)

# 전처리 파이프라인 만들기
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import MinMaxScaler

my_pipeline = Pipeline([('mean_imputer',Imputer(strategy='mean')),
                         ('mm_scaler', MinMaxScaler())])


X_train_fit = my_pipeline.fit(X_train)
X_train_pre = X_train_fit.transform(X_train)
X_test_pre = X_train_fit.transform(X_test)
print(pd.DataFrame(X_train_pre).describe()) # 전처리 상황 확인
y_train_fit = my_pipeline.fit(y_train)
y_train_pre = y_train_fit.transform(y_train)
y_test_pre = y_train_fit.transform(y_test)

# 회귀 모델 만들기

def my_regressor(df_X, df_y):
    from sklearn.linear_model import LinearRegression # 일반 회귀 모델
    from sklearn.linear_model import Ridge, Lasso, ElasticNet # norm 규제 회귀 모델
    from sklearn.linear_model import ARDRegression, BayesianRidge # 베이지안 회귀
    from sklearn.ensemble import RandomForestRegressor # decision tree 앙상블 모델, 배깅
    from xgboost import XGBRegressor # decision tree 앙상블 모델, 부스팅
    from sklearn.model_selection import cross_val_score # model 검증
    
    import ast # convert string to function
    import numpy as np 
    
    
    linear = LinearRegression()
    ridge, lasso, elasticnet = Ridge(), Lasso(), ElasticNet()
    ardr_linear, baysian_ridge = ARDRegression(), BayesianRidge()
    random_forest = RandomForestRegressor()
    xgboost_linear = XGBRegressor()
    
    my_model_list = ['linear', 'ridge', 'lasso', 'elasticnet', 
                     'ardr_linear', 'baysian_ridge', 
                     'random_forest', 'xgboost_linear'] 
    
    score_dic = dict()
    for model_nm in my_model_list:
        scores = cross_val_score(eval(model_nm), df_X, df_y, scoring= "neg_mean_squared_error") # cv default = 5
        rmse_score = np.sqrt(-scores)
        rmse_sm = rmse_score.mean()
        score_dic[model_nm] = rmse_sm
    
    score_dic= sorted(score_dic.items(), key=lambda t : t[1])
    
    return score_dic[0]

    
linear_model_train_score = my_regressor(X_train_pre, y_train_pre)
print(linear_model_train_score)